# Fase 1: Extracción de Datos de API de MoviesDataset
En esta fase se utilizará la API de MoviesDataset para extraer información relevante sobre películas y cortometrajes.

**Requerimientos:**
- Películas desde 1990 hasta la actualidad.
- Géneros: Drama, Comedy, Action, Fantasy, Horror, Mystery, Romance, Thriller.
- Información necesaria:
  - Tipo (corto o película).
  - Nombre.
  - Año y mes de estreno.
  - ID de la película.

**Nota:** Los datos extraídos deberán almacenarse en una lista de tuplas.

---

## Importamos librerías, configuraciones y variables.

In [104]:
%load_ext autoreload
%autoreload 2

from config import *

import sys
sys.path.append('../')

import os
from dotenv import load_dotenv

load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

---
---
## Hacemos llamada a la API

### Definimos variables globales procedentes de nuestro archivo .env

In [105]:
API_URL = os.getenv('API_URL')
API_KEY = os.getenv('API_KEY')
API_HOST = os.getenv('API_HOST')

### Definimos varibales para los bucles que van a realizar todas las llamadas a las apis y así obtener 50 peliculas y cortos por generos.

In [106]:
genres = ['Drama', 'Comedy', 'Action', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller']
tipes = ['movie', 'short']
years_range = [i for i in range(1990,2025)]

In [107]:
def get_registers(movie):
    movie_type = movie['titleType']['id']
    movie_name = movie['titleText']['text']

    movie_year = 0
    try:
        movie_year = movie['releaseDate']['year']
    except:
        movie_year = 0

    movie_month = 0
    try:
        movie_month = movie['releaseDate']['month']
    except:
        movie_month = 0

    movie_year_month = f"{movie_year}-{movie_month}"

    movie_id = movie['id']

    return (movie_type, movie_name, movie_year_month, movie_id)

In [108]:
def get_all_registers(api_url, api_key, api_host, genres_, tipes_, years_range_):
    all_registers = []

    url = api_url

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": api_host
    }

    for tipe in tqdm(tipes_):
        for genre in genres_:
            for year in years_range_:

                querystring = {"genre":genre,"titleType":tipe,"year":year,"sort":"year.incr","endYear":"2024","limit":"50"}

                response = requests.get(url, headers=headers, params=querystring).json()
    
                results = response['results']
    
                all_registers.append(list(map(get_registers, results)))
    

    return all_registers

In [109]:
movie_shorts_list = get_all_registers(API_URL, API_KEY, API_HOST, genres, tipes, years_range)

100%|██████████| 2/2 [03:11<00:00, 95.77s/it] 


### aplanamos nuestra lista

In [112]:
movie_shorts_list_flated = [item for row in movie_shorts_list for item in row]

In [117]:
movie_shorts_list_flated[:4]

[('movie', "S. Lubin's Passion Play", '1903-1', 'tt11166772'),
 ('movie', 'The Story of the Kelly Gang', '1906-12', 'tt0000574'),
 ('movie', 'Robbery Under Arms', '1907-11', 'tt0000615'),
 ('movie', "L'enfant prodigue", '1907-6', 'tt0000591')]

---
---
## guardamos los datos

In [118]:
with open(f'{DATOS_RUTA_BASE}/movie_shorts_list_flated.pkl', 'wb') as f:
    pickle.dump(movie_shorts_list_flated, f)